In [1]:
import json

# Read json file
json_file = open('IOA_COVIDKhoj_data_export.json')
json = json.load(json_file)

# Iterate through json file structure and append each row to a list
# (not the most elegant, but it gets the job done better than json_normalize)
leads = []
for state in json:
    for service in json[state]: 
        for phone_number in json[state][service]:
            lead = {
                'State': state,
                'Service': service,
                'Phone Number': phone_number,
            }
            # Concatenates all the remaining lowest level fields to the dict 
            lead.update(json[state][service][phone_number])
            leads.append(lead)

In [2]:
import pandas as pd
import numpy as np

# Construct a pandas dataframe (this is where the fun begins!)
df = pd.DataFrame(leads)

# Renaming columns
df.rename(columns={
    'desc': 'Description',
    'area': 'Location',
    'city': 'City'
}, inplace=True)

# Remove extra services and merge their details into "Description" column
desc_adders = {
    'remdesivir': 'Medicine name: Remdesivir',
    'fabiflu': 'Medicine name: Fabiflu',
    'tocilizumab': 'Medicine name: Tocilizumab',
    'ambulance': 'Notes: Ambulance Services', 
    'oxygen_bed': 'Notes: Has Oxygen Beds', 
    'icu_bed': 'Notes: Has ICU Bed', 
    'icu_bed with ventilator': 'Has ICU Bed with Ventilator'
}

for key, value in desc_adders.items():
    df['Description'] = df['Description'] + \
        np.where(df['Service'] == key, ' ; ' + value, '')

# Merge phone number columns
df['Phone Number'] += np.where(df['phone'].notnull(), ' ; ' + df['phone'], '')

# Properly format state and city
df['State'] = df['State'].str.title() 
df['City'] = df['City'].str.title()

# Replace values that were already merged, misspelled values, etc.
df.replace({
    'beds': 'Hospital Beds',
    'oxygen': 'Oxygen',
    'remdesivir': 'Medicine', # Added to description
    'food': 'Food', 
    'other medicines': 'Medicine',
    'plasma': 'Blood / Plasma',
    'ambulance': 'Other', # Added to description
    'oxygen_cylinders': 'Oxygen',
    'teleconsultation': 'Telehealth',
    'oxygen_bed': 'Hospital Beds', # Added to description
    'fabiflu': 'Medicine', # Added to description
    'icu_bed': 'Hospital Beds', # Added to description
    'icu_bed with ventilator': 'Hospital Beds', # Added to description
    'only_bed': 'Hospital Beds',
    'tocilizumab': 'Medicine', # Added to description
    'mumbai': 'Mumbai'
}, inplace=True)

# Drop columns that aren't relevant or ones we've already merged
df = df.drop(columns=['firstVerifiedAt', 'phone'])

df['link'].value_counts() # TODO: Revisit
df['pincode'].value_counts() # TODO: Revisit
# TODO: Merge "date" and "time" columns into one "Timestamp of Entry" column
# TODO: Parse "Phone Number" colum with the phonenumbers package
# TODO: Remove "\n" and "undefined" strings from "Description" column
df

,State,Service,Phone Number,Description,name,verified,Location,City,date,link,pincode,time
0,Andhra Pradesh,Hospital Beds,9848940304,Guntur Covid fighter's team: \nnot verified: 8...,UNKNOWN,1619775552320,NaN,NaN,NaN,NaN,NaN,NaN
1,Andhra Pradesh,Oxygen,8712912987,"not verified : 8712912987, 9989153803, 9849122...",UNKNOWN,1620559998910,NaN,NaN,NaN,NaN,NaN,NaN
2,Andhra Pradesh,Oxygen,9350068204,undefined\npossibly available on 02/05/21,Vinay,1619876473368,NaN,NaN,NaN,NaN,NaN,NaN
3,Andhra Pradesh,Oxygen,9966012368,"chinna waltair, vizag, landmark: near reliance...",Rafi,1619671262176,NaN,NaN,NaN,NaN,NaN,NaN
4,Andhra Pradesh,Medicine,7829980066,All India Helpline for Remdesvir \n ; Medicine...,Mylan helpline,1619792907925,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
619,Rajasthan,Telehealth,9552555677,available through phone calls from 4 to 9 PM.s...,Dr. Nikita Rajkondawar,NaN,,Fatehpur,23 April,,,17:47
620,Telangana,Medicine,9367526015,they can arrange in 5-6 days ; Medicine name: ...,"GRS Life Care, Madurai",NaN,,Hyderabad,23 April,"1st Floor, 159 G,Thalai Veethi 2 Cross Street,...",625020,4:58:02 PM
621,West Bengal,Hospital Beds,8743800403,NaN,Keshav Mishra,1619456337087,NaN,NaN,NaN,NaN,NaN,NaN
622,West Bengal,Oxygen,9830083350,Refilling in Baghajatin area\nThey'll test whe...,Prabal Chaudhury,1619864117295,NaN,NaN,NaN,NaN,NaN,NaN
